In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
import nltk
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.tokenize import word_tokenize
from sklearn.calibration import CalibratedClassifierCV

from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV

stops = set(stopwords.words("english"))                  
stemming = PorterStemmer()
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [2]:

def lower_case(df):
    lower= [word.lower() for word in df1.listingText]
    df1.listingText = lower
    return df

def clean_text(text):
    text = re.sub(r'^https?:\/\/.*\r\n[\r\n]*', '', text, flags=re.MULTILINE) 
    text = text.replace(r'\d+', '')
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"[0123456789]",'', text)
    text = re.sub(r"\n",' ', text)
    text = re.sub(r"\r",' ', text)
    text=re.sub(r'[^\w\s]','',text)
    return text

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(stemming.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


#applying upsampling the the minority classes ( helps with accuracy)
def up_sample(df):
    dfmax = df1.unit_type.value_counts()
    dfmax = dfmax.max()
    dfapt = df[df.labelcode ==0]
    dftwnhm = df[df.labelcode ==1]
    StudioApartment = df[df.labelcode ==2]
    Basement = df[df.labelcode ==3]
    Assisted_Living = df[df.labelcode ==4]
    Condo = df[df.labelcode ==5]
    Single_Family_Home = df[df.labelcode ==6]
    Mobile_Home = df[df.labelcode ==7]
    Duplex = df[df.labelcode ==8]
    Triplex = df[df.labelcode ==9]
    Accessory_Unit = df[df.labelcode ==10]
    
    
    dftwnhm = resample(dftwnhm,
                          replace=True,     # sample with replacement
                          n_samples=dfmax,    # to match majority class
                          random_state=123)

    StudioApartment = resample(StudioApartment, 
                           replace=True,     # sample with replacement
                           n_samples=dfmax,    # to match majority class
                           random_state=123)

    Basement = resample(Basement,
                          replace=True,     # sample with replacement
                          n_samples=dfmax,    # to match majority class
                          random_state=123)


    Assisted_Living = resample(Assisted_Living, replace=True,     
                           n_samples=dfmax,   
                           random_state=123)

    Condo = resample(Condo, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)

    Single_Family_Home = resample(Single_Family_Home, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)

    Mobile_Home = resample(Mobile_Home, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)
    
    Duplex = resample(Duplex, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)
    
    Triplex = resample(Triplex, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)
    
    Accessory_Unit = resample(Accessory_Unit, replace=True,     
                       n_samples=dfmax,   
                       random_state=123)
    
    df= pd.concat([dfapt,dftwnhm, StudioApartment, Basement,
                   Assisted_Living,Condo,
                   Single_Family_Home,Mobile_Home, 
                   Duplex,Triplex,Accessory_Unit])
    
    return df


In [3]:
# reading an cleaning data as well as preparing it for modeling 
df = pd.read_csv("coded_unit_type.csv")
df1 = df[["unit_type", "listingText"]]
df1.listingText = df1.listingText.astype(str)
df1.listingText = lower_case(df1.listingText)
df1['listwords'] = df1['listingText'].apply(lambda x: clean_text(x))
df1['listwords'] = df1['listwords'].apply(stemSentence)

df1['labelcode'] = df1['unit_type'].factorize()[0]
df1 = up_sample(df1)

In [4]:
df1.unit_type.value_counts()

Accessory Unit        81
Townhome              81
 Mobile Home          81
Studio Apartment      81
Apartment             81
Basement              81
 Duplex               81
Triplex               81
Condo                 81
Single Family Home    81
Assisted Living       81
Name: unit_type, dtype: int64

In [23]:
category_id_df = df1[['unit_type', 'labelcode']].drop_duplicates().sort_values('labelcode')
category_id_df

,unit_type,labelcode
0,Apartment,0
28,Townhome,1
167,Studio Apartment,2
121,Basement,3
31,Assisted Living,4
126,Condo,5
78,Single Family Home,6
93,Mobile Home,7
101,Duplex,8
115,Triplex,9


In [ ]:
df1

In [6]:
#Benchmarking multiple models. 


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=5, 
                        norm='l2', 
                        encoding='latin-1', 
                        ngram_range=(1, 2), 
                        stop_words='english')

features = tfidf.fit_transform(df1.listwords).toarray()
labels = df1.labelcode
features.shape



models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.figure(figsize=(16, 6))
plt.show()

NameError: name 'features' is not defined

In [93]:
cv_df.groupby('model_name').accuracy.mean().sort_values()

model_name
AdaBoostClassifier               0.270455
DecisionTreeClassifier           0.540909
KNeighborsClassifier             0.783021
RandomForestClassifier           0.839572
GaussianProcessClassifier        0.902072
MultinomialNB                    0.908957
SVC                              0.917279
LogisticRegression               0.925535
GaussianNB                       0.925869
LinearSVC                        0.935695
MLPClassifier                    0.935695
QuadraticDiscriminantAnalysis    0.944987
Name: accuracy, dtype: float64

In [7]:
#pipeline of feature engineering and model for linearsvc

model = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),
                  ('clf', CalibratedClassifierCV(LinearSVC(class_weight="balanced")))])


X_train, X_test, y_train, y_test = train_test_split(df1.listwords, df1.unit_type, test_size=0.3, random_state=5)

parameters = {'tfidf__ngram_range': [(1, 1), (1, 2),(2,2)],
              'tfidf__use_idf': (True, False),
              'tfidf__sublinear_tf': (True, False),
              'tfidf__norm': ('l1', 'l2'), 
              'clf__base_estimator__C': [1.0, 10, 100, 1000]}
             
gs_clf_svm = GridSearchCV(model, parameters,cv =5, n_jobs=-2)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)

0.9406099518459069


In [24]:
category_id_df

,unit_type,labelcode
0,Apartment,0
28,Townhome,1
167,Studio Apartment,2
121,Basement,3
31,Assisted Living,4
126,Condo,5
78,Single Family Home,6
93,Mobile Home,7
101,Duplex,8
115,Triplex,9


In [60]:
#extracting probabilities 
probs = gs_clf_svm.predict_proba(y_test)
#extracting predictions
probs = pd.DataFrame(probs)
preds = gs_clf_svm.predict(y_test)
#putting together 
output_df = probs
output_df = pd.DataFrame(output_df)
output_df['predicted_label'] = preds

#adding the actual names to the label probabilities 
output_df.columns = [ 
                     "Apartment",
                     "Townhome",
                     "Studio Apartment",
                     "Basement", 
                     "Assisted Living",
                     "Condo", 
                     "Single Family Home",
                     "Mobile Home", 
                     "Duplex", 
                     "Triplex",
                     "Accessory Unit", 
                     "predicted_label"]

#reordering for interpretability
output_df = output_df[["predicted_label",
                       "Apartment",
                     "Townhome",
                     "Studio Apartment",
                     "Basement", 
                     "Assisted Living",
                     "Condo", 
                     "Single Family Home",
                     "Mobile Home", 
                     "Duplex", 
                     "Triplex",
                     "Accessory Unit"
                      ]]

In [61]:
output_df

,predicted_label,Apartment,Townhome,Studio Apartment,Basement,Assisted Living,Condo,Single Family Home,Mobile Home,Duplex,Triplex,Accessory Unit
0,Apartment,0.010125,0.012034,0.007760,0.717111,0.017965,0.013753,0.010513,0.045301,0.019480,0.138046,0.007913
1,Apartment,0.010125,0.012034,0.007760,0.717111,0.017965,0.013753,0.010513,0.045301,0.019480,0.138046,0.007913
2,Single Family Home,0.000700,0.030461,0.001849,0.015969,0.022780,0.003473,0.000204,0.798500,0.005542,0.118904,0.001620
3,Single Family Home,0.000700,0.030461,0.001849,0.015969,0.022780,0.003473,0.000204,0.798500,0.005542,0.118904,0.001620
4,Apartment,0.010377,0.012078,0.007966,0.729945,0.018438,0.014111,0.010810,0.040736,0.016221,0.122481,0.016838
5,Single Family Home,0.000700,0.030461,0.001849,0.015969,0.022780,0.003473,0.000204,0.798500,0.005542,0.118904,0.001620
6,Apartment,0.010377,0.012078,0.007966,0.729945,0.018438,0.014111,0.010810,0.040736,0.016221,0.122481,0.016838
7,Apartment,0.010568,0.012123,0.008525,0.485348,0.020988,0.106109,0.006132,0.192949,0.008906,0.140379,0.007974
8,Apartment,0.010125,0.012034,0.007760,0.717111,0.017965,0.013753,0.010513,0.045301,0.019480,0.138046,0.007913
9,Apartment,0.010125,0.012034,0.007760,0.717111,0.017965,0.013753,0.010513,0.045301,0.019480,0.138046,0.007913


In [62]:
def get_confidence(df):
    levels = ["Confident", "Fairly Confident", "Low Confidence", "Very Unconfident"]
    logical = []
    df = df.round(4)
    df.iloc[:, 1:11] = df.iloc[:, 1:11].mul(100)
    df['max_vals_per_row'] = df.iloc[:, 1:11].max(axis=1)
    for i in df.max_vals_per_row:
        if i >= 95.5:
            logical.append(levels[0])
        elif i >= 85.5:
            logical.append(levels[1])
        elif i >= 70.5:
            logical.append(levels[2])
        else:
            logical.append(levels[3])
    df["prediction_confidence"] = logical
    return df

In [63]:
output_df = get_confidence(output_df)

In [64]:
output_df

,predicted_label,Apartment,Townhome,Studio Apartment,Basement,Assisted Living,Condo,Single Family Home,Mobile Home,Duplex,Triplex,Accessory Unit,max_vals_per_row,prediction_confidence
0,Apartment,1.01,1.20,0.78,71.71,1.80,1.38,1.05,4.53,1.95,13.80,0.0079,71.71,Low Confidence
1,Apartment,1.01,1.20,0.78,71.71,1.80,1.38,1.05,4.53,1.95,13.80,0.0079,71.71,Low Confidence
2,Single Family Home,0.07,3.05,0.18,1.60,2.28,0.35,0.02,79.85,0.55,11.89,0.0016,79.85,Low Confidence
3,Single Family Home,0.07,3.05,0.18,1.60,2.28,0.35,0.02,79.85,0.55,11.89,0.0016,79.85,Low Confidence
4,Apartment,1.04,1.21,0.80,72.99,1.84,1.41,1.08,4.07,1.62,12.25,0.0168,72.99,Low Confidence
5,Single Family Home,0.07,3.05,0.18,1.60,2.28,0.35,0.02,79.85,0.55,11.89,0.0016,79.85,Low Confidence
6,Apartment,1.04,1.21,0.80,72.99,1.84,1.41,1.08,4.07,1.62,12.25,0.0168,72.99,Low Confidence
7,Apartment,1.06,1.21,0.85,48.53,2.10,10.61,0.61,19.29,0.89,14.04,0.0080,48.53,Very Unconfident
8,Apartment,1.01,1.20,0.78,71.71,1.80,1.38,1.05,4.53,1.95,13.80,0.0079,71.71,Low Confidence
9,Apartment,1.01,1.20,0.78,71.71,1.80,1.38,1.05,4.53,1.95,13.80,0.0079,71.71,Low Confidence


In [65]:
output_df.to_excel("output_unittype.xlsx")